In [ ]:
!pip uninstall -y libact
!pip install git+https://github.com/IINemo/libact.git@seq

In [1]:
!pip uninstall -y actleto
!pip install -e ../../packages/active_learning_toolbox/
#!pip install git+https://github.com/IINemo/active_learning_toolbox.git

Uninstalling actleto-0.1.0:
  Successfully uninstalled actleto-0.1.0
Obtaining file:///notebook/projects/biomed_ie/packages/active_learning_toolbox


  Running setup.py develop for actleto
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
!pip uninstall -y text_selector
!pip install -e ../text_selector/

Uninstalling text-selector-0.1.0a0:
  Successfully uninstalled text-selector-0.1.0a0


In [3]:
!jupyter nbextension install --py --symlink --sys-prefix text_selector

Installing /notebook/projects/biomed_ie/packages/text_selector/text_selector/static -> text_selector
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable text_selector --py --sys-prefix
    


In [4]:
!jupyter nbextension enable --py --sys-prefix text_selector

Enabling notebook extension text_selector/extension...
      - Validating: OK


In [1]:
import sys


sys.path.append('../../../bert_sequence_tagger/src/')

# Initializer

In [2]:
import collections, pandas as pd, numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

from libact.models import LogisticRegression as LibActLogReg
from libact.query_strategies import UncertaintySampling

from actleto import ActiveLearner, ActiveLearnerUiWidget, make_libact_strategy_ctor

# OLD

In [2]:
# Loading the 20newsgroups dataset
train_dataset = fetch_20newsgroups(subset='train')
test_dataset = fetch_20newsgroups(subset='test')

In [3]:
# Preparing features
vectorizer = TfidfVectorizer(min_df=3, max_df=0.5, sublinear_tf=True)

X_train = vectorizer.fit_transform(train_dataset.data)
Y_train = train_dataset.target

X_test = vectorizer.transform(test_dataset.data)
Y_test = test_dataset.target

In [ ]:
clf = LogisticRegression(solver='lbfgs')
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
Y_pred_proba = clf.predict_proba(X_test)
print('Accuracy', accuracy_score(Y_test, Y_pred))
print('F1 macro', f1_score(Y_test, Y_pred, average='macro'))

In [ ]:
# We choose 20 random examples as seed set for active learning.
Y_seed = Y_train.copy().astype('O')
unknown_indexes = np.random.randint(Y_seed.shape[0], size = Y_seed.shape[0] - 20)
Y_seed[unknown_indexes] = None

In [6]:
# Creating query strategy. The strategy selects examples for annotations from unlabeled dataset.
# We use standard uncertainty sampling strategy from libact and wrap it into adaptor.
active_learn_alg_ctor = make_libact_strategy_ctor(lambda trn_ds:
                                                  UncertaintySampling(trn_ds,
                                                                      model = LibActLogReg()))

# Creating ActiveLearning object that implements AL logic.
active_learner = ActiveLearner(active_learn_alg_ctor = active_learn_alg_ctor,
                               y_dtype = 'int',
                               X_full_dataset = X_train, 
                               y_full_dataset = Y_seed,
                               X_test_dataset = X_test,
                               y_test_dataset = Y_test,
                               model_evaluate = LibActLogReg(), # We use logreg from libact for choosing samples.
                               eval_metrics = [accuracy_score, f1_score],
                               rnd_start_steps = 0)

# Creaing X_helper object for visualization dataset for humans.
X_helper = pd.DataFrame(data={ 'text' : train_dataset.data })

/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [ ]:
# This try-catch block is needed to stop autosave thread in case we invoke the cell multiple times.
try:
    if active_learn_ui:
        active_learn_ui.stop()
except NameError:
    pass

# Creaing the active learner widget itself and configure it with active_learner, X_helper.
# The default visualizer will preview the visualize_columns with VisualizerTextArea.
active_learn_ui = ActiveLearnerUiWidget(active_learner = active_learner, 
                                        X_helper = X_helper,
                                        display_feature_table = False,
                                        drop_labels = [],
                                        visualize_columns = ['text'],
                                        y_labels = { label : i for i, label in enumerate(train_dataset.target_names) },
                                        save_path = './20ng_active_dump.npy',
                                        save_time = 120)

active_learn_ui

# New

In [3]:
txts = [
    "The human TCF-1 gene encodes a nuclear DNA-binding protein uniquely expressed in normal and neoplastic T-lineage lymphocytes .",
    "The anti-inflammatory drugs sodium salicylate and aspirin inhibited the activation of NF-kappa B, which further explains the mechanism of action of these drugs .",
    "Expression of IL-1alpha by HTLV-I productively infected cells may be important in the hypercalcemia , osteolytic bone lesions , neutrophilia , elevation of C-reactive protein, and fever frequently seen in patients with HTLV-I-induced adult T-cell leukemia/lymphoma .",
    "These data may have implications for the stringent control of IL-4 expression in T cells .",
    "PI3-kinase activity was an upstream activator of PKB/Akt because the PI3-kinase inhibitor LY294002 blocked both constitutive PKB/Akt and factor-dependent PKB/Akt activity .",
    "Cell-type-specific regulation of the human <B-DNA> tumor <I-DNA> necrosis <I-DNA> factor <I-DNA> alpha <I-DNA> gene <I-DNA> in B <B-cell_type> cells <I-cell_type> and T <B-cell_type> cells <I-cell_type> by NFATp <B-protein> and ATF-2/JUN <B-protein> .",
    "The IL-2 <B-protein> enhancer contains a functional motif named CD28 <B-DNA> response <I-DNA> element <I-DNA> ( CD28RE <B-DNA> ) that serves a role as a target for mitogenic T cell activation signals .",
    "Here , we describe the effect of 383-bp <B-DNA> ( <I-DNA> delta383 <I-DNA> ) <I-DNA> and <I-DNA> 99-bp <I-DNA> ( <I-DNA> delta99 <I-DNA> ) <I-DNA> sequences <I-DNA> containing the PYBF-binding <B-DNA> site <I-DNA> on transcription from various globin and non-globin promoters , using a transient assay with the cat <B-DNA> reporter <I-DNA> gene <I-DNA> in murine <B-cell_line> erythroleukemia <I-cell_line> ( <I-cell_line> MEL <I-cell_line> ) <I-cell_line> cells <I-cell_line> , a cell line with abundant PYBF <B-protein> activity .",
    "Since the co-expression with a dominant negative c-Fos <B-protein> abolished the responsiveness to TPA , we conclude that activated transcription of the DRA <B-DNA> gene <I-DNA> depends on interactions between the X2 <B-DNA> box <I-DNA> and NF-X2 <B-DNA> , which contains c-Fos <B-protein> .",
    "To test this hypothesis , 4-h NK cytolysis assays were performed on IFN-gamma-treated <B-cell_line> human <I-cell_line> cells <I-cell_line> infected with an Ad5 E1A <B-DNA> deletion mutant , dl343 , or transfected with the Ad5 <B-DNA> E1A <I-DNA> gene <I-DNA> .",
    "Taken together , these findings suggest that PWM <B-protein> is able to initiate an intracytoplasmic signalling cascade and EGR-1 <B-DNA> induction in normal <B-cell_type> human <I-cell_type> B <I-cell_type> cells <I-cell_type> .",
    "As several inflammatory <B-protein> cytokines <I-protein> and oxidative stresses are involved in the pathogenesis of HIV-1 infection , we investigated the inhibitory effects of cepharanthine on tumor necrosis factor alpha ( TNF-alpha <B-protein> ) - and phorbol 12-myristate 13-acetate ( PMA ) -induced HIV-1 replication in chronically <B-cell_line> infected <I-cell_line> cell <I-cell_line> lines <I-cell_line> .",
    "The authors provide the results of studying 1 , 25-dihydroxyvitamin D3 ( calcitriol ) in lymphocytes <B-cell_type> of children with rickets and rickets-like diseases .",
    "These results suggest that HIV-TF1 <B-protein> is involved in transcriptional regulation of HIV-1 .",
    "Associated with these findings , the levels of p21 <B-protein> ( <I-protein> WAF1/CIP1 <I-protein> ) <I-protein> and p27 <B-RNA> ( <I-RNA> KIP1 <I-RNA> ) <I-RNA> mRNA <I-RNA> and protein increased in these cells .",
    "The gene encoding the Epstein-Barr <B-protein> virus <I-protein> ( <I-protein> EBV <I-protein> ) <I-protein> -specific <I-protein> dUTPase <I-protein> was amplified from virus <B-DNA> DNA <I-DNA> by PCR .",
    "In vivo studies in rats utilized 2 mg/kg IV bolus doses of liposomal MPL compared to drug in solution .",
    "Only through considered evaluation of clinical evidence may clinicians determine which therapies should remain novelties and which should become an accepted part of the armamentarium .",
    "The levels of cAMP-dependent <B-protein> protein <I-protein> kinase <I-protein> and Rap1 <B-protein> were assessed by Western blot analysis , immunostaining , and computer-assisted imaging .",
    "Oncogenicity of human <B-cell_line> papillomavirus- <I-cell_line> or <I-cell_line> adenovirus-transformed <I-cell_line> cells <I-cell_line> correlates with resistance to lysis by natural <B-cell_type> killer <I-cell_type> cells <I-cell_type> .",
    "Because the primed <B-cell_line> T <I-cell_line> cells <I-cell_line> contain higher levels of nuclear <B-protein> NF-ATp <I-protein> , capable of binding to P <B-DNA> elements <I-DNA> of the IL-4 <B-DNA> and <I-DNA> IL-13 <I-DNA> promoters <I-DNA> , than do freshly-isolated <B-cell_type> T <I-cell_type> cells <I-cell_type> , the NF-AT-binding <B-DNA> P <I-DNA> elements <I-DNA> are attractive candidates to mediate the coordinate expression of these two cytokine <B-DNA> genes <I-DNA> .",
    "While the maternally <B-DNA> derived <I-DNA> X <I-DNA> chromosome <I-DNA> was preferentially active in the asymptomatic twin , the paternal <B-DNA> X <I-DNA> chromosome <I-DNA> was active in the other , affected twin and was found in her hemizygotic nephew .",
    "Studies of the actions of platelet-activating <B-protein> factor <I-protein> have centered mainly around neutrophils <B-cell_type> , monocytes <B-cell_type> , and platelets <B-cell_type> .",
    "Stat3gamma <B-protein> is derived from Stat3alpha <B-protein> by limited proteolysis and lacks the carboxyl-terminal <B-protein> portion <I-protein> of Stat3alpha <B-protein> .",
    "Examination of IkappaB <B-protein> alpha <I-protein> regulation revealed that TNF-alpha <B-protein> -mediated degradation of IkappaB <B-protein> alpha <I-protein> in both memory <B-cell_type> and <I-cell_type> naive <I-cell_type> T <I-cell_type> cells <I-cell_type> from the elderly was severely impaired , thus contributing to the lowered induction of the observed NFkappaB <B-protein> .",
    "This provides the hormone with an interesting therapeutic potential .",
    "We now demonstrate that v-src <B-DNA> expression correlates with nuclear expression of a kappa <B-protein> B <I-protein> binding <I-protein> complex <I-protein> similar to that induced by phorbol ester and ionomycin , as detected by electrophoretic mobility shift assay using a variety of kappa <B-DNA> B <I-DNA> sites <I-DNA> .",
    "We therefore examined the effects of EC <B-cell_type> on T <B-protein> cell <I-protein> nuclear <I-protein> factors <I-protein> known to regulate IL-2 <B-protein> transcription , including c-jun <B-protein> and c-fos-two <B-protein> components <I-protein> of the transcription <B-protein> factor <I-protein> AP-1 <B-protein> , NFAT <B-protein> , and others .",
    "Contrary to the results in Jurkat <B-cell_line> cells <I-cell_line> , PDTC did not inhibit tumor <B-protein> necrosis <I-protein> factor-alpha <I-protein> -induced NF <B-protein> kappaB <I-protein> activation in astrocytes <B-cell_type> ; instead PDTC itself induces NF <B-protein> kappaB <I-protein> activation in astrocytes <B-cell_type> , and this may be related to scavenging of endogenously produced NO by the PDTC iron complex .",
    "To identify the molecular basis for specificity and inducibility , the chromatin <B-DNA> configuration of the human <B-DNA> B7.1 <I-DNA> gene <I-DNA> was examined in intact nuclei from various cell types .",
    "Human umbilical vein ECs <B-cell_type> were pretreated with alpha-tcp before stimulation with known agonists of monocyte adhesion : IL-1 <B-protein> ( 10 ng/ml ) , LPS ( 10 ng/ml ) , thrombin <B-protein> ( 30 U/ml ) , or PMA ( 10 nM ) .",
    "The 5 <B-DNA> ' <I-DNA> sequences <I-DNA> up to nucleotide -120 of the human and murine IL-16 <B-protein> genes share > 84 % sequence homology and harbor <B-DNA> promoter <I-DNA> elements <I-DNA> for constitutive and inducible transcription in T <B-cell_type> cells <I-cell_type> .",
    "C3 transferase and cytochalasin D did not alter the expression levels of monocyte-binding <B-protein> receptors <I-protein> on endothelial <B-cell_type> cells <I-cell_type> , but did inhibit clustering of E-selectin <B-protein> , ICAM-1 <B-protein> , and VCAM-1 <B-protein> on the cell surface induced by monocyte adhesion or cross-linking <B-protein> antibodies <I-protein> .",
    "In this study , we isolated the genomic <B-DNA> DNA <I-DNA> for human <B-DNA> IRF-2 <I-DNA> gene <I-DNA> , determined the intron-exon structure of the human <B-DNA> IRF-2 <I-DNA> gene <I-DNA> , mapped the major <B-DNA> transcription <I-DNA> initiation <I-DNA> site <I-DNA> , identified a number of potential regulatory elements in the 5'-flanking <B-DNA> region <I-DNA> , and localized the IRF-2 <B-DNA> gene <I-DNA> on human <B-DNA> chromosome <I-DNA> 4 <I-DNA> .",
    "CD30 <B-protein> is a cell-surface <B-protein> receptor <I-protein> that can augment lymphocyte activation and survival through its ability to induce the transcription <B-protein> factor <I-protein> NF-kappaB <B-protein> .",
    "Despite the dual specificity of HTLV-1 Tax <B-protein> for IkappaBalpha <B-protein> and IkappaBbeta <B-protein> at the protein level , Tax <B-protein> selectively stimulates NF-kappaB <B-protein> -directed transcription of the IkappaBalpha <B-DNA> gene <I-DNA> .",
    "The agent , 17 beta-estradiol , but not 17 alpha-estradiol , significantly reduced PMNs <B-cell_type> chemotaxis to FMLP in a dose-dependent manner ( control vs estrogen 10 ( -10 ) - ( -6 ) M , P < 0.05 ) .",
    "In PHA-activated <B-cell_line> PBL <I-cell_line> , no age-related differences were observed in c-fos or jun B mRNA expression .",
    "Control of IL-5 <B-protein> production may be effective for the management of allergic diseases .",
    "NFAT1 <B-protein> possesses two transactivation <B-DNA> domains <I-DNA> whose sequences are not conserved in the other NFAT-family <B-protein> proteins <I-protein> , and a conserved DNA-binding <B-DNA> domain <I-DNA> that mediates the recruitment of cooperating nuclear <B-protein> transcription <I-protein> factors <I-protein> even when it is expressed in the absence of other regions of the protein .",
    "Expression of Spi-1 <B-protein> in these cells inhibited this ability to differentiate and rescued the cells from the apoptotic cell death program normally induced upon hEpo <B-protein> withdrawal .",
    "Pretreatment of cultured <B-cell_type> human <I-cell_type> ECs <I-cell_type> with the PPARalpha <B-protein> activators fenofibrate or WY14643 inhibited TNF-alpha-induced <B-protein> VCAM-1 <I-protein> in a time- and concentration-dependent manner , an effect not seen with PPARgamma <B-protein> activators .",
    "The molecular mechanism ( s ) responsible for this phenomenon have not been clearly elucidated .",
    "GC efficiently suppressed IL-5 <B-protein> synthesis of T-cell clones activated via either T-cell <B-protein> receptor <I-protein> ( TCR <B-protein> ) or IL-2 <B-protein> receptor <I-protein> ( IL-2R <B-protein> ) .",
    "Alveolar macrophages play an important role in host defense and in other types of inflammatory processes in the lung .",
    "To limit expression of a transduced HIV-1 <B-DNA> long <I-DNA> terminal <I-DNA> repeat <I-DNA> ( <I-DNA> LTR <I-DNA> ) <I-DNA> -IFNA2 <I-DNA> ( the new approved nomenclature for IFN <B-DNA> genes <I-DNA> is used throughout this article ) hybrid <B-DNA> gene <I-DNA> to the HIV-1-infected <B-cell_line> cells <I-cell_line> , HIV-1 <B-DNA> LTR <I-DNA> was modified .",
    "Characterization of the CD48 <B-DNA> gene <I-DNA> demonstrates a positive element that is specific to Epstein-Barr <B-cell_type> virus-immortalized <I-cell_type> B-cell <I-cell_type> lines <I-cell_type> and contains an essential NF-kappa <B-DNA> B <I-DNA> site <I-DNA> .",
    "Such observations indicate that IL-4 <B-protein> Stat <I-protein> has the same functional <B-protein> domain <I-protein> for both receptor coupling and dimerization .",
    "Double-stimulation was used to demonstrate that , in a T <B-cell_line> lymphocytic <I-cell_line> cell <I-cell_line> line <I-cell_line> ( CEM <B-cell_line> ) , phorbol myristate acetate ( PMA ) rapidly induced NF-kappa <B-protein> B <I-protein> through a signaling pathway which did not involve reactive oxygen species ( ROS ) and was different from the activation triggered by either H2O2 or tumor <B-protein> necrosis <I-protein> factor-alpha <I-protein> ( TNF-alpha <B-protein> ) .",
    "Mutations in the TSC2 <B-DNA> gene <I-DNA> : analysis of the complete coding sequence using the protein truncation test ( PTT ) .",
]

tags = [
    'protein',
    'DNA',
    'cell_type',
    'cell_line',
    'RNA'
]


import pandas as pd

df = pd.DataFrame([[e] for e in txts], columns=['texts'])

In [4]:
class SeqAnnotationVisualizer:
    def __init__(self, tags):
        self._tags = tags
        self._widgets = {}
        
    def _callback(self, index):
        self._answers[index] = self._widgets[index].res
        
    def init(self, dataframe, answers):
        self._dataframe = dataframe
        self._answers = answers
        
    def __call__(self, index):
        widget = TextSelectorWidget(txt=self._dataframe.iloc[index, 0], tags=self._tags, 
                                    callback=lambda *args,**kwargs: self._callback(index), 
                                    colors=['#33ff99', '#66b2ff', '#ff9999', '#ffff66', '#C0C0C0'])
        if self._answers[index] is not None:
            widget.res = self._answers[index]
            
        self._widgets[index] = widget
        return (widget,)

In [5]:
set(['#33ff99', '#66b2ff', '#ff9999', '#ffff66', '#C0C0C0'])

{'#33ff99', '#66b2ff', '#C0C0C0', '#ff9999', '#ffff66'}

In [6]:
tags = ['cell_type', 'DNA', 'cell_line', 'RNA', 'protein']

In [7]:
from text_selector.widget import Widget as TextSelectorWidget
from ipywidgets import HBox, Button, Label, VBox


from actleto.annotator.annotator_widget import AnnotatorWidget


ann_widget = AnnotatorWidget(df, 
                             answers=None,
                             max_examples=5,
                             current_position=0,
                             visualize_columns=[], 
                             drop_labels=[],
                             y_labels=None,
                             #y_labels={i : e for i, e in enumerate(tags)},
                             visualizer=SeqAnnotationVisualizer(tags=tags),
                             display_feature_table=False)

In [ ]:
# TODO:
# +0. Где callback?
# 1. Убрать prev-next
# 2. Пофиксить аннотацию в разных аннотаторах 
# 3. В разметку добавляется NaN:NaN если просто кликнуть на кноку Add
# 4. Почему разными цветами всегда светится ?
# 5. Почему кнопка reset не работает?
# +6. Как создать виджет с уже имеющейся разметкой?

In [8]:
ann_widget

AnnotatorWidget(children=(VBox(children=(HBox(children=(Button(description='Prev', style=ButtonStyle()), Butto…

In [22]:
answ = ann_widget.get_answers()
answ

array([list([{'start': 4, 'end': 19, 'tag': 'DNA'}, {'start': 39, 'end': 57, 'tag': 'protein'}, {'start': 103, 'end': 123, 'tag': 'protein'}]),
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None], dtype=object)

In [14]:
answ = ann_widget.get_answers()
import pickle 

with open('my_answers', 'wb') as f:
    pickle.dump(answ, f)

In [38]:
X_train = pd.DataFrame([[e] for e in txts], columns=['texts'])
y_seed = [None] * len(txts)
y_seed[0] = {'0' : {(0, 5) : 'protein'}}

In [9]:
import sys

sys.path.append('../../../bert_sequence_tagger/src/')

In [10]:
def find_in_between(offsets, start, end):
    res = []
    for i, offset in enumerate(offsets):
        if start <= offset and offset <= end:
            res.append(i)
    return res


def convert_y_to_bio_format(X, y):
    final_res = []
    for i, sent_y in enumerate(y):
        sent = X[i]
        offsets = []
        curr_offset = 0
        for index, word in enumerate(sent.split(' ')):
            offsets.append(curr_offset)
            curr_offset += len(word) + 1
        
        good_ys = ['O'] * len(sent)
        for w_y in sent_y:
            positions = find_in_between(offsets, w_y['start'], w_y['end'])
            good_ys[positions[0]] = 'B-' + w_y['tag']
            
            for pos in positions[1:]:
                good_ys[pos] = 'I-' + w_y['tag']
        
        final_res.append(good_ys)
    
    return final_res


def create_helper(X_train):
    X_helper = pd.DataFrame([' '.join(e) for e in X_train], columns=['texts'])
    return X_helper


def convert_y_to_dict_format(X, y):
    dict_annots = []
    for sent_x, sent_y in zip(X, y):
        offsets = []
        curr_offset = 0
        for index, word in enumerate(sent_x):
            offsets.append(curr_offset)
            curr_offset += len(word) + 1

        sent_dict_annots = []
        start_offset = -1
        last_offset = -1
        entity_tag = ''
        for i, tag in enumerate(sent_y):
            if tag.split('-')[0] == 'O':
                if start_offset != -1:
                    sent_dict_annots.append({'tag' : entity_tag, 
                                            'start' : start_offset, 
                                            'end' : last_offset})
                start_offset = -1
                
            if tag.split('-')[0] == 'B':
                if start_offset != -1:
                    sent_dict_annots.append({'tag' : entity_tag, 
                                            'start' : start_offset, 
                                            'end' : last_offset})
                
                start_offset = offsets[i]
                entity_tag = tag.split('-')[1]
                last_offset = offsets[i] + len(sent_x[i])
            elif tag.split('-')[0] == 'I':
                last_offset = offsets[i] + len(sent_x[i])
        
        if start_offset != -1:
            sent_dict_annots.append({'tag' : entity_tag,
                             'start' : start_offset,
                             'end' : last_offset})
        
        dict_annots.append(sent_dict_annots)
    
    return dict_annots

In [10]:
bio_format = convert_y_to_bio_format([df.iloc[0, 0]], [answ[0]])

NameError: name 'answ' is not defined

In [76]:
list(zip(df.iloc[0, 0].split(' '), bio_format[0]))

[('The', 'O'),
 ('human', 'B-DNA'),
 ('TCF-1', 'I-DNA'),
 ('gene', 'I-DNA'),
 ('encodes', 'O'),
 ('a', 'O'),
 ('nuclear', 'O'),
 ('DNA-binding', 'B-protein'),
 ('protein', 'I-protein'),
 ('uniquely', 'O'),
 ('expressed', 'O'),
 ('in', 'O'),
 ('normal', 'O'),
 ('and', 'O'),
 ('neoplastic', 'O'),
 ('T-lineage', 'B-protein'),
 ('lymphocytes', 'I-protein'),
 ('.', 'O')]

In [11]:
from libact.base.interfaces import ProbabilisticModel
from libact.base.dataset import Dataset
from libact.query_strategies import RandomSampling

import torch
from torch.utils.data import RandomSampler, SequentialSampler, DataLoader

from bert_sequence_tagger.bert_utils import create_loader_from_flair_corpus

import numpy as np
from sklearn.model_selection import train_test_split
from collections.abc import Iterable  



class LibActNN(ProbabilisticModel):
    def __init__(self, 
                 model_ctor, 
                 trainer_ctor,
                 batch_size=16,
                 bs_pred=256, 
                 retrain_epochs=3, 
                 iter_retrain=1,
                 train_from_scratch=True, 
                 valid_ratio=0.25,
                 string_input=True):
        self._model_ctor = model_ctor
        self._trainer_ctor = trainer_ctor
        self._model = None
        self._trainer = None
        self._batch_size = batch_size
        self._bs_pred = bs_pred
        self._retrain_epochs = retrain_epochs
        self._batch_size = batch_size
        self._iter_retrain = iter_retrain
        self._train_from_scratch = train_from_scratch
        self._valid_ratio = valid_ratio
        self._string_input = string_input
        
        self._iter = 0
        
    def _predict_core(self, X):
        if self._string_input:
            X = [sent.split(' ') for sent in X]
            
        torch.cuda.empty_cache()
        valid_sampler = SequentialSampler(X)
        valid_dataloader = DataLoader(X, sampler=valid_sampler, 
                                      batch_size=self._bs_pred, 
                                      collate_fn=lambda inpt: inpt)
        return self._model.predict(valid_dataloader)
    
    def predict_proba(self, X):
        return np.asarray(self._predict_core(X)[1]).reshape(-1, 1)
    
    def predict(self, X):
        return self._predict_core(X)[0]
    
    def train(self, libact_dataset, new_indexes=None):
        torch.cuda.empty_cache()
        collate_fn = lambda inpt: tuple(zip(*inpt))
        
        if (new_indexes is not None) and (self._iter % self._iter_retrain) != 0:
            libact_dataset = Dataset([libact_dataset.data[i][0] for i in new_indexes], 
                                     [libact_dataset.data[i][1] for i in new_indexes])
            n_epochs = 1
        else:
            n_epochs = self._retrain_epochs
            
        X, y = libact_dataset.format_sklearn()
        if self._string_input:
            y = convert_y_to_bio_format(X, y)
        
        if self._valid_ratio > 0.:
            X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=self._valid_ratio)
            
            valid_data = list(zip(X_valid, y_valid))
            valid_sampler = SequentialSampler(valid_data)
            valid_dataloader = DataLoader(valid_data, 
                                           sampler=valid_sampler, 
                                           batch_size=self._bs_pred,
                                           collate_fn=collate_fn)
        else:
            X_train, y_train = X, y
            valid_dataloader = None
        
        train_data = list(zip(X_train, y_train))
        #print(train_data)
        train_sampler = RandomSampler(train_data)
        train_dataloader = DataLoader(train_data, 
                                      sampler=train_sampler, 
                                      batch_size=self._batch_size,
                                      collate_fn=collate_fn)

        
        if (self._model is None) or self._train_from_scratch:
            self._model = self._model_ctor()
            self._trainer = self._trainer_ctor(self._model, len(X_train), 
                                               train_dataloader, valid_dataloader)
            import gc
            gc.collect()
            torch.cuda.empty_cache()
            
        self._trainer.train(self._retrain_epochs)
        
        self._iter += 1
        
    def score(self):
        pass
    
    
# def prepare_X_for_adapter(X):
#     return [sent.split(' ') for sent in X]

    
# class AdaptorForALWidget:
#     def __init__(self, *args, **kwargs):
#         super().__init__(*args, **kwargs)
        
#     def predict_proba(self, X):
#         return super().predict_proba(prepare_X_for_adapter(X))
    
#     def predict(self, X):
#         return super(self, X).predict(prepare_X_for_adapter(X))
    
#     def train(self, libact_dataset, new_indexes=None):
        
#         return super().train(libact_dataset, new_indexes):

/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/tensorflow/pyth

In [ ]:
import pickle 

with open('my_answers', 'rb') as f:
    my_answ = pickle.load(f)
    
my_answ

In [12]:
from libact.base.dataset import Dataset


class LibActNNWrapper:
    def __init__(self, *args, **kwargs):
        super.__init__(*args, **kwargs)
        
    def train(self, libact_dataset, new_indexes=None):
        new_dataset = Dataset()
        for entry in libact_dataset.get_entries():
            if entry[1] is not None:
                new_dataset
            else:
                new_dataset.data.append(entry)

In [13]:
from bert_sequence_tagger.bert_utils import make_bert_tag_dict_from_flair_corpus, prepare_flair_corpus


def prepare_corpus(corpus):
    X, y = [], []
    for X_i, y_i in prepare_flair_corpus(corpus):
        X.append(X_i)
        y.append(y_i)
    
    return X, y


from flair.datasets import ColumnCorpus


data_folder = '../../workdir/genia/conll/'
corpus = ColumnCorpus(data_folder, {0 : 'text', 1 : 'ner'},
                                train_file='Genia4ERtask1.iob2',
                                test_file='Genia4EReval1.iob2',
                                dev_file='Genia4EReval1.iob2')

tags_vals, tag2idx = make_bert_tag_dict_from_flair_corpus(corpus)
    
X_train, y_train = prepare_corpus(corpus.train)
X_test, y_test = prepare_corpus(corpus.test)

2019-11-17 22:24:16,844 Reading data from ../../workdir/genia/conll
2019-11-17 22:24:16,845 Train: ../../workdir/genia/conll/Genia4ERtask1.iob2
2019-11-17 22:24:16,846 Dev: ../../workdir/genia/conll/Genia4EReval1.iob2
2019-11-17 22:24:16,847 Test: ../../workdir/genia/conll/Genia4EReval1.iob2


In [14]:
from bert_sequence_tagger import BertForTokenClassificationCustom, SequenceTaggerBert, ModelTrainerBert
from bert_sequence_tagger.bert_utils import get_parameters_without_decay
from bert_sequence_tagger.metrics import f1_entity_level

from pytorch_transformers import BertTokenizer, AdamW

from torch.optim.lr_scheduler import ReduceLROnPlateau

CACHE_DIR = '../../workdir/cache'

# MAX_LEN = 150
BATCH_SIZE = 32
#BATCH_SIZE = 16
MAX_LEN = 100
#BATCH_SIZE = 100
#BATCH_SIZE_PRED = 1200 # 1600
#BATCH_SIZE_PRED = 1200 # 1600
BATCH_SIZE_PRED = 1500
PRED_BATCH_SIZE = BATCH_SIZE_PRED
random_state = 2019

EARLY_STOPPING = 1
MAX_RETRAIN_EPOCHS = 30
#MAX_RETRAIN_EPOCHS = 15
MAX_N_EPOCHS = MAX_RETRAIN_EPOCHS

N_SAMPLES_PER_AL_ITER = 30
N_AL_PASSES = 5
N_AL_ITERATIONS = 25
# N_AL_PASSES = 1
# N_AL_ITERATIONS = 1

BASE_LR = 5e-5
LEARNING_RATE = BASE_LR

VALIDATION_RATIO = 0.25

MAX_TO_ANNEAL = 3
ANNEAL_FACTOR = 0.5
PATIENCE = 2
WEIGHT_DECAY = 0.01

SEED_POSITIVE = 10
SEED_RANDOM = 40

BIO_BERT = '../../workdir/bio_bert/torch2/'
OUTPUT_FILE_PATH = '../../workdir/experiments/19_biobert/'
#OUTPUT_FILE_PATH = '../workdir/experiments/debug'

# index2tag = ['[PAD]', 'O', 'B-protein', 'I-protein']
# tag2index = {t : i for i, t in enumerate(index2tag)}


BERT_TOKENIZER = BertTokenizer.from_pretrained('bert-base-cased', 
                                               cache_dir=CACHE_DIR, 
                                               do_lower_case=False)


def model_ctor():
    model = BertForTokenClassificationCustom.from_pretrained(BIO_BERT,
                                                             cache_dir=CACHE_DIR, 
                                                             num_labels=len(tag2idx)).cuda()

    seq_tagger = SequenceTaggerBert(model, BERT_TOKENIZER, idx2tag=tags_vals, tag2idx=tag2idx)

    return seq_tagger

def trainer_ctor(seq_tagger, corpus_len, train_dataloader, val_dataloader):
    optimizer = AdamW(get_parameters_without_decay(seq_tagger._bert_model),
                      lr=LEARNING_RATE, betas=(0.9, 0.999), 
                      eps=1e-6, weight_decay=0.01, correct_bias=True)

    lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=ANNEAL_FACTOR, patience=PATIENCE)

    trainer = ModelTrainerBert(seq_tagger, 
                               optimizer, 
                               lr_scheduler,
                               train_dataloader, 
                               val_dataloader,
                               update_scheduler='ee',
                               keep_best_model=True,
                               restore_bm_on_lr_change=True,
                               max_grad_norm=1.,
                               validation_metrics=[f1_entity_level],
                               decision_metric=lambda metrics: metrics[0],
                               smallest_lr=LEARNING_RATE / (MAX_TO_ANNEAL**(1./ANNEAL_FACTOR) + 0.1))

    return trainer

In [15]:
y_train_dict = convert_y_to_dict_format(X_train, y_train)
X_helper = create_helper(X_train)

In [16]:
import random

sample_size = 100
random_sample = random.sample(list(range(len(y_train))), sample_size)

y_seed_dict = [None for _ in range(len(y_train_dict))]

for elem in random_sample:
    y_seed_dict[elem] = y_train_dict[elem]

In [22]:
# X_helper = create_helper(X_train)
# y_dict = convert_y_to_dict_format([X_train[0]], [y_train[0]])

In [126]:
# print(X_helper.iloc[0, 0])
# y_dict

IL-2 gene expression and NF-kappa B activation through CD28 requires reactive oxygen production by 5-lipoxygenase .


[[{'tag': 'DNA', 'start': 0, 'end': 9},
  {'tag': 'protein', 'start': 25, 'end': 35},
  {'tag': 'protein', 'start': 55, 'end': 59},
  {'tag': 'protein', 'start': 99, 'end': 113}]]

In [127]:
# bio_format = convert_y_to_bio_format([X_helper.iloc[0, 0]], [y_dict[0]])
# list(zip(X_train[0], bio_format[0]))

{'tag': 'DNA', 'start': 0, 'end': 9}
{'tag': 'protein', 'start': 25, 'end': 35}
{'tag': 'protein', 'start': 55, 'end': 59}
{'tag': 'protein', 'start': 99, 'end': 113}


[('IL-2', 'B-DNA'),
 ('gene', 'I-DNA'),
 ('expression', 'O'),
 ('and', 'O'),
 ('NF-kappa', 'B-protein'),
 ('B', 'I-protein'),
 ('activation', 'O'),
 ('through', 'O'),
 ('CD28', 'B-protein'),
 ('requires', 'O'),
 ('reactive', 'O'),
 ('oxygen', 'O'),
 ('production', 'O'),
 ('by', 'O'),
 ('5-lipoxygenase', 'B-protein'),
 ('.', 'O')]

In [17]:
active_learn_alg_ctor = make_libact_strategy_ctor(lambda trn_ds:
                                                  UncertaintySampling(trn_ds,
                                                                      model = LibActNN(model_ctor=model_ctor, 
                                                                                       trainer_ctor=trainer_ctor,
                                                                                       valid_ratio=0.)),
                                                  max_samples_number=40)

# Creating ActiveLearning object that implements AL logic.
active_learner = ActiveLearner(active_learn_alg_ctor = active_learn_alg_ctor,
                               y_dtype = 'int',
                               #X_full_dataset = X_train, 
                               #y_full_dataset = y_seed,
                               X_full_dataset = X_helper.texts.tolist(),
                               y_full_dataset = y_seed_dict,
                               #eval_metrics = [accuracy_score, f1_score],
                               rnd_start_steps = 1)

# Creaing X_helper object for visualization dataset for humans.
#X_helper = pd.DataFrame(data={ 'text' : train_dataset.data })

# Todo convert y_seed to format that can be handled by Widget
# Todo convert results to format that can be used for training

active_learner.start()

Epoch: 100%|██████████| 3/3 [00:05<00:00,  1.84s/it]


In [18]:
# This try-catch block is needed to stop autosave thread in 
#case we invoke the cell multiple times.
try:
    if active_learn_ui:
        active_learn_ui.stop()
except NameError:
    pass

# Creaing the active learner widget itself and configure 
#it with active_learner, X_helper.
# The default visualizer will preview 
#№the visualize_columns with VisualizerTextArea.
active_learn_ui = ActiveLearnerUiWidget(active_learner=active_learner,
                                        X_helper=X_helper,
                                        display_feature_table=False,
                                        drop_labels = [],
                                        y_labels=None,
                                        save_path='./20ng_active_dump.npy',
                                        save_time=120, 
                                        visualizer=SeqAnnotationVisualizer(tags=tags))

active_learn_ui

ActiveLearnerUiWidget(children=(HBox(children=(Button(description='Next iteration', style=ButtonStyle()), Labe…

Epoch: 100%|██████████| 3/3 [00:05<00:00,  1.86s/it]
